In [44]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Kombinasi 5 :
- Delete duplicate
- Impute missing value using iterative imputer
- Outlier handling with log tranformation
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [45]:
df = pd.read_csv('../../Punya Andi/UFC_kombinasi5.csv')
df.head()

,B_Height_cms,B_Weight_lbs,B_win_by_TKO_Doctor_Stoppage,B_avg_LEG_att,gender,B_win_by_Submission,B_age,B_avg_opp_HEAD_landed,B_avg_CLINCH_att,R_avg_opp_TD_att,...,B_avg_CTRL_time(seconds),R_avg_KD,B_avg_opp_SUB_ATT,B_avg_SIG_STR_pct,B_avg_opp_TD_pct,B_avg_opp_GROUND_att,B_avg_opp_TOTAL_STR_att,B_avg_opp_TD_att,R_Reach_cms,B_Reach_cms
0,5.817944,5.707110,0.0,2.197225,1,0.000000,3.806662,3.198673,1.770706,1.749200,...,4.892520,0.000000,0.693147,0.357674,0.152721,1.998096,4.887997,0.559616,5.802722,190.50
1,5.802722,5.602119,0.0,2.018023,1,0.000000,3.828641,3.336937,1.897901,1.918759,...,3.609439,0.405465,0.124703,0.332278,0.011419,0.436237,4.531339,1.042373,5.832938,177.80
2,5.817944,5.655992,0.0,1.609438,1,0.000000,3.850148,3.167583,0.000000,0.693147,...,2.844182,0.000000,0.000000,0.373286,0.000000,0.223144,4.594251,0.000000,5.825469,190.50
3,5.795024,5.707110,0.0,1.486564,1,0.000000,3.850148,2.237213,2.554680,1.539164,...,4.238174,0.019852,0.060625,0.578499,0.099280,0.594707,3.971176,0.965081,5.825469,175.26
4,5.763622,5.525453,0.0,2.227707,1,1.386294,3.970292,3.329550,2.395985,2.537534,...,5.351532,0.277868,0.000244,0.323143,0.384970,1.917504,4.843083,1.543805,5.771566,170.18


In [46]:
X = df.drop(['B_Reach_cms'], axis=1)
y = df['B_Reach_cms']

In [47]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [48]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Define the parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'leaf_size': [10, 20, 30, 40, 50],
    'p': [1, 2]
}

# Create the KNN model
knn = KNeighborsRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='r2')

# Fit the data to perform grid search
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best R-squared Score:", grid_search.best_score_)


Best Parameters: {'leaf_size': 10, 'metric': 'manhattan', 'n_neighbors': 11, 'p': 1, 'weights': 'distance'}
Best R-squared Score: 0.7254632349854798


In [49]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Evaluate the best model using 5-fold cross-validation
cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='r2')

# Calculate the mean R-squared
mean_r2 = cv_scores.mean()
print("Mean R-squared:", mean_r2)

Mean R-squared: 0.7254632349854798


In [50]:
# Calculate the RMSE using 5-fold cross-validation
cv_rmse = np.sqrt(np.abs(cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')))

# Calculate the mean RMSE
mean_rmse = cv_rmse.mean()
print("Mean RMSE:", mean_rmse)

Mean RMSE: 5.409536591076991


In [51]:
df_test = pd.read_csv('../../regression_kaggle/UFC_kombinasi5.csv')
df_test = df_test.drop(['B_Reach_cms'], axis=1, errors='ignore')
df_test_id = df_test['id']
df_test = df_test.drop(['id'], axis=1)
df_test = scaler.transform(df_test)
y_pred = grid_search.best_estimator_.predict(df_test)
submission = pd.DataFrame({'id': df_test_id, 'B_Reach_cms': y_pred})
submission.to_csv('pred_kombinasi4_knn.csv', index=False)